In [1]:
import nltk
from nltk.tag.stanford import StanfordNERTagger
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords
from nltk import pos_tag, ne_chunk
from nltk.tokenize import word_tokenize
from nltk.chunk import conlltags2tree, tree2conlltags

sw = stopwords.words("english")

path_to_model = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\classifiers\english.all.3class.distsim.crf.ser.gz'
path_to_jar = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\stanford-ner.jar'
tagger = StanfordNERTagger(path_to_model, path_to_jar = path_to_jar)

In [2]:
from gensim.models.word2vec import LineSentence
from gensim.models import Phrases
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

#visualization libraries
import pyLDAvis
import pyLDAvis.gensim

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
import os
import pandas as pd
import numpy as np
import codecs
import itertools as it
from bs4 import BeautifulSoup
import warnings
import pickle
from collections import Counter
import re
import datetime
import string
import time

java_path = 'C:/Program Files/Java/jdk-10.0.1/bin/java.exe'
os.environ['JAVAHOME'] = java_path

In [4]:
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\inves\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\inves\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\inves\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [65]:
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn.utils import resample, shuffle
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

from sklearn.base import clone
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from xgboost import XGBClassifier

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import model_from_json

from keras import utils

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\ops.py:858: DeprecationWarning: builtin type EagerTensor has no __module__ a

In [111]:
#import corpus/docket texts from html to pandas DataFrame
def grab_docket_text():
    files = []
    #get all .html files in the folder (all docket files are in .html)
    for file in os.listdir('docket_texts/Test/'):
        if file.endswith('.html'):
            files.append(os.path.join('docket_texts/Test/', file))

    df_docket_texts = pd.DataFrame()
    
    for i in range(len(files)): #gather all docket texts
    #for i in [0, 1]: #for testing purposes
        
        content = codecs.open(files[i], 'r', 'utf-8').read()
        #use beautiful soup to get the case ID
        soup = BeautifulSoup(content, 'lxml')
        case_id = str(soup.find_all('h3'))    
        bookmark1 = case_id.find('CASE #:') + len('CASE #:')
        bookmark2 = case_id.find('</h3>')
        case_id = case_id[bookmark1:bookmark2]

        #use pandas to grab tables in the html files
        docket_tables = pd.read_html(content)

        #error checking: gotta do this because there's different length of docket_list/
        #usually docket texts are in docket_list[3], but not always
        n = 0
        while docket_tables[n].isin(['Docket Text']).sum().sum() == 0:
            #print(n, docket_tables[n].isin(['Docket Text']).sum().sum())
            n += 1
                        
        print(i, files[i])
        #print(docket_tables[n].head())

        #docket_tables[n] is the docket text table
        new_header = docket_tables[n].iloc[0]
        docket_tables[n] = docket_tables[n][1:]
        docket_tables[n].columns = new_header
        
        docket_tables[n]['#'] = pd.to_numeric(docket_tables[n]['#'],
                                              downcast = 'signed', errors = 'coerce')
        docket_tables[n]['Date Filed'] = pd.to_datetime(docket_tables[n]['Date Filed'])
        docket_tables[n]['Case ID'] = case_id

        df_docket_texts = pd.concat([df_docket_texts, docket_tables[n]])
    #reorder a column
    cols = list(df_docket_texts.columns)
    df_docket_texts = df_docket_texts[[cols[-1]] + cols[:-1]]
    
    print('current docket text table size/shape: {}'.format(df_docket_texts.shape))
    return df_docket_texts

In [112]:
%%time
df = grab_docket_text()
docket_original = list(df['Docket Text'])
for i in range(5):
    print('docket text {}'.format(i))
    print(docket_original[i], '\n')

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\bs4\builder\_lxml.py:250: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  self.parser.feed(markup)


0 docket_texts/Test/SDNY_00015.html
1 docket_texts/Test/SDNY_00037.html
2 docket_texts/Test/SDNY_00039.html
3 docket_texts/Test/SDNY_00097.html
4 docket_texts/Test/SDNY_00130.html
5 docket_texts/Test/SDNY_00659.html
6 docket_texts/Test/SDNY_00792.html
7 docket_texts/Test/SDNY_01179.html
8 docket_texts/Test/SDNY_02023.html
9 docket_texts/Test/SDNY_04407.html
10 docket_texts/Test/SDNY_05271.html
11 docket_texts/Test/SDNY_06374.html
12 docket_texts/Test/SDNY_06513.html
13 docket_texts/Test/SDNY_07898.html
14 docket_texts/Test/SDNY_09696.html
current docket text table size/shape: (1804, 4)
docket text 0
FILING ERROR - DEFICIENT PLEADING - FILED AGAINST PARTY ERROR COMPLAINT against All Defendants. (Filing Fee $ 400.00, Receipt Number 0208-11793625)Document filed by Majid Soueidan.(Rosen, Samuel) Modified on 1/5/2016 (pc). (Entered: 01/04/2016) 

docket text 1
FILING ERROR PDF ERROR CIVIL COVER SHEET filed. (Rosen, Samuel) Modified on 1/5/2016 (pc). (Entered: 01/04/2016) 

docket text 2
REQ

In [12]:
%%time
path_to_model = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\classifiers\english.all.3class.distsim.crf.ser.gz'
path_to_jar = r'C:\Users\inves\AppData\Local\Programs\Python\Python35\Lib\site-packages\nltk\stanford-ner-2018-02-27\stanford-ner.jar'
tagger = StanfordNERTagger(path_to_model, path_to_jar = path_to_jar)

output = []
#length = 10
length = len(docket_original)
for i in range(length):
    org_str = []
    name_str = []
    stripped_str1 = []
    stripped_str2 = []
    tokens = nltk.tokenize.word_tokenize(docket_original[i])
    #print(tokens)
    for label, token in zip(tagger.tag(tokens), tokens):
        #print(label)
        if label[1] == 'ORGANIZATION':
            org_str.append(label[0])
            stripped_str1.append('-ORG-')
        elif label[1] == 'PERSON':
            name_str.append(label[0])
            stripped_str1.append('-NAME-')
        else:
            stripped_str1.append(token)
            stripped_str2.append(token)
    
    output.append([docket_original[i],
                   ' '.join(org_str),
                   ' '.join(name_str),
                   ' '.join(stripped_str1),
                   ' '.join(stripped_str2)])

Wall time: 50min 37s


In [13]:
NER_df = pd.DataFrame(output, columns = ['Original Docket Text', 'Organization Portion', 'Name Portion', 
                                         'Identifying Org and Name', 'Stripped Org and Name'])

In [14]:
print(NER_df.shape)
NER_df.head()

(1804, 5)


,Original Docket Text,Organization Portion,Name Portion,Identifying Org and Name,Stripped Org and Name
0,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,,Majid Soueidan Rosen Samuel,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...
1,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,,Rosen Samuel,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...
2,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,,BRAD PEDERSEN ROBERT J. KELLY NELSON OBUS WILL...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,,Majid Soueidan Rosen Samuel,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...
4,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,,Samuel Kenneth Rosen,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...


In [15]:
#to save
filename = 'docket_texts/test/DT/basic_df.pickle'
with open(filename, 'wb') as handle: 
    pickle.dump(NER_df, handle, protocol = pickle.HIGHEST_PROTOCOL)

### Don't have to start from the very beginning

In [113]:
filename = 'docket_texts/test/DT/basic_df.pickle'

In [114]:
#to load
with open(filename, 'rb') as handle:
    NER_df = pickle.load(handle)
    
new_df = NER_df.copy()
print(new_df.shape)
print(new_df.drop_duplicates().shape)
new_df.head()

(1804, 5)
(1797, 5)


,Original Docket Text,Organization Portion,Name Portion,Identifying Org and Name,Stripped Org and Name
0,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,,Majid Soueidan Rosen Samuel,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...
1,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,,Rosen Samuel,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...
2,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,,BRAD PEDERSEN ROBERT J. KELLY NELSON OBUS WILL...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,,Majid Soueidan Rosen Samuel,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...
4,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,,Samuel Kenneth Rosen,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...


In [115]:
### Normalize
def normalize(docket_original):

    url_regex1 = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}\[\]]+|\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\))+(?:\([^\s()]*?\([^\s()]+\)[^\s()]*?\)|\([^\s]+?\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    url_regex2 = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    date_regex = '(\d{1,2}[\/ ](\d{2}|January|Jan|February|Feb|March|Mar|April|Apr|May|May|June|Jun|July|Jul|August|Aug|September|Sep|October|Oct|November|Nov|December|Dec)[\/ ]\d{2,4})'
    punct_regex = r"[^a-zA-Z0-9]"
    num_regex = "\d+"
    extraspace_regex = " +"
    docket_original = docket_original.split(' ')
    docket_normalized = [text.lower() for text in docket_original]
    docket_nourl1 = [re.sub(url_regex1, "url", text) for text in docket_normalized]
    docket_nourl2 = [re.sub(url_regex2, "url", text) for text in docket_nourl1]
    docket_nodate = [re.sub(date_regex, "date", text) for text in docket_nourl2]
    docket_nopunct = [re.sub(punct_regex, " ", text) for text in docket_nodate]
    docket_nonum = [re.sub(num_regex, " ", text) for text in docket_nopunct]
    #new, remove if the last words are entered date
    docket_noextraspace = re.sub(extraspace_regex, " ", ' '.join(docket_nonum))
    docket_no_entereddate = docket_noextraspace.replace('entered date', '')
    
    return docket_no_entereddate

In [116]:
t0 = time.time()
new_df['normalized'] = new_df['Stripped Org and Name'].apply(lambda x: normalize(x))
print('took {} seconds'.format(time.time() - t0))

took 0.6707525253295898 seconds


In [117]:
def tokenize(docket_noextraspace):
    return word_tokenize(docket_noextraspace)

In [118]:
t0 = time.time()
new_df['tokenize'] = new_df['normalized'].apply(tokenize)
print('took {} seconds'.format(time.time() - t0))

took 0.4059562683105469 seconds


In [119]:
def stopwords(docket_tokenized):  
    return [word for word in docket_tokenized if word not in sw]

In [120]:
t0 = time.time()
new_df['nostop'] = new_df['tokenize'].apply(stopwords)
print('took {} seconds'.format(time.time() - t0))

took 0.14419198036193848 seconds


In [121]:
#this is updated, comparing mannual_topics_20180828 and manual_topics_20180911 and manual_topics_20180919
manual_topics_df = pd.read_csv('manual_topics_20180919.csv') 
manual_topics_df = manual_topics_df.apply(lambda x: x.astype(str).str.lower())
manual_topics_dict = manual_topics_df.to_dict('list')
for topic in manual_topics_dict.keys():
    manual_topics_dict[topic] = [keyword for keyword in manual_topics_dict[topic] if keyword != 'nan']

In [122]:
#output a list of topics
def mannual_topic_assignment(text):
    #text = text.split()
    #print(text)
    output = []
    for topic in manual_topics_dict.keys():
        for keyword in manual_topics_dict[topic]:
            if ' '.join(text).find(keyword) != -1:
                output.append(topic)
    #print(output)
    #if there's motion to dismiss, then remove noaction
    if ' '.join(text).find('motion dismiss') != -1 and 'NoAction' in output:
        output = list(set(output))
        output.remove('NoAction')
    return ', '.join(set(output))

In [123]:
docket_texts_output = list(new_df['nostop'])

docket_texts_output_DT = []
topics_DT = []
NoAction = []

for text in docket_texts_output:
    topic = mannual_topic_assignment(text)
    #print(text, topic)
    if topic != '':
        docket_texts_output_DT.append('')
        topics_DT.append(topic)
        if 'NoAction' in topic:
            NoAction.append(1)
        else:
            NoAction.append(0)
    else:
        docket_texts_output_DT.append(text)
        topics_DT.append('')
        NoAction.append(0)

In [124]:
new_df['DT Topics'] = pd.Series(topics_DT)
new_df['NoAction'] = pd.Series(NoAction)
new_df['Removed unnecessary POS & vocab DT'] = pd.Series(docket_texts_output_DT)
print(new_df.shape)
new_df.head()

(1804, 11)


,Original Docket Text,Organization Portion,Name Portion,Identifying Org and Name,Stripped Org and Name,normalized,tokenize,nostop,DT Topics,NoAction,Removed unnecessary POS & vocab DT
0,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,,Majid Soueidan Rosen Samuel,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,filing error deficient pleading filed against ...,"[filing, error, deficient, pleading, filed, ag...","[filing, error, deficient, pleading, filed, pa...","Notices, Complaints",0,
1,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,,Rosen Samuel,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,filing error pdf error civil cover sheet filed...,"[filing, error, pdf, error, civil, cover, shee...","[filing, error, pdf, error, civil, cover, shee...",Notices,0,
2,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,,BRAD PEDERSEN ROBERT J. KELLY NELSON OBUS WILL...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,request for issuance of summons as to breeze e...,"[request, for, issuance, of, summons, as, to, ...","[request, issuance, summons, breeze, eastern, ...","Service of Process, Complaints",0,
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,,Majid Soueidan Rosen Samuel,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...,rule corporate disclosure statement no corpora...,"[rule, corporate, disclosure, statement, no, c...","[rule, corporate, disclosure, statement, corpo...",,0,"[rule, corporate, disclosure, statement, corpo..."
4,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,,Samuel Kenneth Rosen,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,notice to attorney regarding deficient pleadi...,"[notice, to, attorney, regarding, deficient, p...","[notice, attorney, regarding, deficient, plead...","Notices, Complaints, Motions",0,


In [27]:
#for verification purposes
new_df.to_excel('something.xlsx', index = False)

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\pandas\io\excel.py:784: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [125]:
def lemm(docket_nostop):
    return ' '.join([WordNetLemmatizer().lemmatize(word, pos='v') for word in docket_nostop])

In [126]:
t0 = time.time()
new_df['lemmed'] = new_df['Removed unnecessary POS & vocab DT'].apply(lemm)
print('took {} seconds'.format(time.time() - t0))
new_df.shape

took 0.0156402587890625 seconds


(1804, 12)

In [127]:
def trigram_transform(texts):
    bigram_model_filepath = 'docket_texts/train/DT/bigram_model_newsop2'
    bigram_model = Phrases.load(bigram_model_filepath)
    
    trigram_model_filepath = 'docket_texts/train/DT/trigram_model_newsop2'
    trigram_model = Phrases.load(trigram_model_filepath)
    
    
    display = False
    texts = str(texts)
    trigram_output = ''
    #print(texts)

    remove_trigram = ['calendar_day', 'court_notice_intend', 'minute_entry_proceeding_hold', 'court_reportertranscriber_abovecaptioned_matter',
                      'redaction_calendar_day', 'rule_statement', 'obtain_pacer', 'may_obtain_pacer', 'reportertranscriber_abovecaptioned_matter',
                      'redact_transcript_deadline', 'send_chamber', "official_transcript_notice_give", "notice_intent_request", "proceed_hold", 
                      "fee_receipt_number", "civil_procedure", "pursuant_frcp", "official_transcript_conference", 
                      "purchase_reportertranscriber_deadline_release", "et_al", "mail_chamber", "transcript_restriction", "redaction_transcript", 
                      "transcript_view_public_terminal", "transcript_make_remotely", "associated_et_al", "electronically_available_public_without", 
                      "genesys_id", "release_transcript_restriction", "adar_bay", "redaction_request_due", "new_york", "official_transcript_conference", 
                      "transcript_make_remotely", "transcript_proceeding_conference_hold", "redaction_transcript",
                      'affidavit_jr._c.p.a', 'corporate_parent', 'certain_underwriter', 'federal_rule_civil_procedure', 'redaction_request', 
                      'official_transcript', 'rule_disclosure', 'rule_corporate_disclosure', 'place_vault', 'public_without_redaction_calendar', 
                      'purchase_deadline_release_transcript', 'transcript_proceeding_hold', 'transcript_remotely_electronically_available',
                      'minute_entry_hold', 'discovery_hear_hold', 'jury_trial_hold', "sign_judge",'place_vault']

    if texts == None:
        return None
    
    unigram_review = []
    for word in texts.split():
        unigram_review.append(word)
    if display:
        print('Uni: ', unigram_review)
    bigram_review = bigram_model[unigram_review]
    if display:
        print('Bi: ', bigram_review)
    trigram_review = trigram_model[bigram_review]
    if display:
        print('Tri: ', trigram_review)
    trigram_review = [phrase for phrase in trigram_review if phrase not in remove_trigram]
    if display:
        print('Tri removed: ', trigram_review)
    trigram_output += ' '.join(trigram_review)
    
    return trigram_output

In [128]:
docket_phrase2 = list(new_df['lemmed'])
docket_phrase2[:10]

['',
 '',
 '',
 'rule corporate disclosure statement corporate parent document file',
 '',
 '',
 '',
 '',
 '',
 '']

In [173]:
docket_phrase3 = [trigram_transform(text) for text in docket_phrase2]
new_df['phrases'] = pd.Series(docket_phrase3)
new_df.shape

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\gensim\models\phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


(1804, 14)

In [174]:
new_df[new_df['DT Topics'] == ''][['lemmed', 'phrases']].head()

,lemmed,phrases
3,rule corporate disclosure statement corporate ...,rule corporate_disclosure statement document file
13,civil cover sheet file,civil_cover_sheet file
15,rule corporate disclosure statement identify c...,rule corporate_disclosure statement identify d...
64,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...
65,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...


In [176]:
new_df['phrases'].iloc[3]

'rule corporate_disclosure statement document file'

In [177]:
new_df.columns

Index(['Original Docket Text', 'Organization Portion', 'Name Portion',
       'Identifying Org and Name', 'Stripped Org and Name', 'normalized',
       'tokenize', 'nostop', 'DT Topics', 'NoAction',
       'Removed unnecessary POS & vocab DT', 'lemmed', 'phrases', 'cleaned'],
      dtype='object')

In [178]:
ss = [3, 13, 15, 1463]
for i in ss:
    print(i)
    print('orignal text: ')
    print(new_df['Original Docket Text'].iloc[i], '\n')
    print('org and name removed: ')
    print(new_df['Stripped Org and Name'].iloc[i], '\n')
    print('normalized: ')
    print(new_df['normalized'].iloc[i], '\n')
    print('stopwords removed: ')
    print(' '.join(new_df['nostop'].iloc[i]), '\n')
    print('after lemmetization: ')
    print(new_df['lemmed'].iloc[i], '\n')
    print('after phrase modeling: ')
    print(new_df['phrases'].iloc[i], '\n')

3
orignal text: 
RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Corporate Parent. Document filed by Majid Soueidan.(Rosen, Samuel) (Entered: 01/04/2016) 

org and name removed: 
RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No Corporate Parent . Document filed by . ( , ) ( Entered : 01/04/2016 ) 

normalized: 
rule corporate disclosure statement no corporate parent document filed by   

stopwords removed: 
rule corporate disclosure statement corporate parent document filed 

after lemmetization: 
rule corporate disclosure statement corporate parent document file 

after phrase modeling: 
rule corporate_disclosure statement document file 

13
orignal text: 
CIVIL COVER SHEET filed. (Rosen, Samuel) (Entered: 01/05/2016) 

org and name removed: 
CIVIL COVER SHEET filed . ( , ) ( Entered : 01/05/2016 ) 

normalized: 
civil cover sheet filed   

stopwords removed: 
civil cover sheet filed 

after lemmetization: 
civil cover sheet file 

after phrase modeling: 
civil_cover_sheet file 

15
orignal t

In [179]:
#not sure if this is necessary as 'cleaned' is not used
remove_pos = list(pd.read_excel('NLP_to_be_removed.xlsx', sheetname = 0, header = None)[0])
remove_word = list(pd.read_excel('NLP_to_be_removed.xlsx', sheetname = 1, header = None)[0])
remove_trigram = list(pd.read_excel('NLP_to_be_removed.xlsx', sheetname = 2, header = None)[0])

In [180]:
def remove_unnecessary(text):
    new_text = []
    text = text.split(' ')
    for word in text:
        if word not in remove_word and word not in remove_trigram:
            new_text.append(word)
    return ' '.join(new_text)

In [181]:
new_df['cleaned'] = new_df['phrases'].apply(remove_unnecessary)
print(new_df.shape)
new_df.head()

(1804, 14)


,Original Docket Text,Organization Portion,Name Portion,Identifying Org and Name,Stripped Org and Name,normalized,tokenize,nostop,DT Topics,NoAction,Removed unnecessary POS & vocab DT,lemmed,phrases,cleaned
0,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,,Majid Soueidan Rosen Samuel,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,filing error deficient pleading filed against ...,"[filing, error, deficient, pleading, filed, ag...","[filing, error, deficient, pleading, filed, pa...","Notices, Complaints",0,,,,
1,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,,Rosen Samuel,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,filing error pdf error civil cover sheet filed...,"[filing, error, pdf, error, civil, cover, shee...","[filing, error, pdf, error, civil, cover, shee...",Notices,0,,,,
2,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,,BRAD PEDERSEN ROBERT J. KELLY NELSON OBUS WILL...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,request for issuance of summons as to breeze e...,"[request, for, issuance, of, summons, as, to, ...","[request, issuance, summons, breeze, eastern, ...","Service of Process, Complaints",0,,,,
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,,Majid Soueidan Rosen Samuel,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...,RULE 7.1 CORPORATE DISCLOSURE STATEMENT . No C...,rule corporate disclosure statement no corpora...,"[rule, corporate, disclosure, statement, no, c...","[rule, corporate, disclosure, statement, corpo...",,0,"[rule, corporate, disclosure, statement, corpo...",rule corporate disclosure statement corporate ...,rule corporate_disclosure statement document file,rule corporate_disclosure
4,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,,Samuel Kenneth Rosen,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,notice to attorney regarding deficient pleadi...,"[notice, to, attorney, regarding, deficient, p...","[notice, attorney, regarding, deficient, plead...","Notices, Complaints, Motions",0,,,,


In [182]:
filename = 'docket_texts/test/DT/basic_cleaned2.pickle'
#to save
with open(filename, 'wb') as handle: 
    pickle.dump(new_df, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [183]:
print('shape before dedupe: {}'.format(new_df.shape))
print('shape after dedupe: {}'.format(new_df[['Original Docket Text', 'normalized', 'DT Topics', 'lemmed', 'phrases']].drop_duplicates().shape))

shape before dedupe: (1804, 14)
shape after dedupe: (1797, 5)


In [184]:
dedup_df = new_df[['Original Docket Text', 'normalized', 'lemmed', 'phrases', 'DT Topics', 'NoAction']].drop_duplicates()

In [56]:
#export to check
dedup_df.to_excel('test_tempfile.xlsx', index = False)

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\pandas\io\excel.py:784: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [185]:
#some text is oddly long
dedup_df['Original Docket Text'].iloc[1793]

"Appeal Record Sent to USCA (Electronic File). Certified Indexed record on Appeal Electronic Files for (127 in 1:07-cv-09633-JSR) Certificate of Service Other, filed by Merrill Lynch & Co., Inc., (10 in 1:07-cv-09696-JSR-JLC, 41 in 1:07-cv-09633-JSR) Notice of Change of Address, filed by James Eastman, (354 in 1:07-cv-09633-JSR) Certificate of Service Other filed by State Teachers Retirement System of Ohio, (219 in 1:07-cv-09633-JSR) Stipulation and Order of Dismissal, (53 in 1:07-cv-09633-JSR) MOTION to Dismiss the Consolidated Amended Class Action Complaint and to Strike Certain Allegations. filed by Merrill Lynch, Pierce, Fenner & Smith Incorporated, Merrill Lynch Capital Trust III, Merrill Lynch Capital Trust II, Merrill Lynch Capital Trust I, Merrill Lynch & Co., Inc., (353 in 1:07-cv-09633-JSR) Affidavit in Support of Motion,, filed by State Teachers Retirement System of Ohio, (64 in 1:07-cv-09633-JSR, 20 in 1:07-cv-09696-JSR-JLC) Notice of Appearance, filed by Alberto Cribiore, 

Separate things into actions and unknowns

In [186]:
dedup_df.head()

,Original Docket Text,normalized,lemmed,phrases,DT Topics,NoAction
0,FILING ERROR - DEFICIENT PLEADING - FILED AGAI...,filing error deficient pleading filed against ...,,,"Notices, Complaints",0
1,FILING ERROR PDF ERROR CIVIL COVER SHEET filed...,filing error pdf error civil cover sheet filed...,,,Notices,0
2,REQUEST FOR ISSUANCE OF SUMMONS as to BREEZE-E...,request for issuance of summons as to breeze e...,,,"Service of Process, Complaints",0
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,rule corporate disclosure statement no corpora...,rule corporate disclosure statement corporate ...,rule corporate_disclosure statement document file,,0
4,***NOTICE TO ATTORNEY REGARDING DEFICIENT PLEA...,notice to attorney regarding deficient pleadi...,,,"Notices, Complaints, Motions",0


In [231]:
deteremined = dedup_df[dedup_df['DT Topics'] != '']
undeteremined = dedup_df[dedup_df['DT Topics'] == '']

In [232]:
deteremined['Action'] = deteremined['NoAction'].map({0: 'Y', 1: 'N'})
deteremined.tail()

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Original Docket Text,normalized,lemmed,phrases,DT Topics,NoAction,Action
1799,Transmission of Notice of Appeal to the Distri...,transmission of to the district judge re in cv...,,,"NoAction, Notices, Motions",1,N
1800,Appeal Record Sent to USCA (Electronic File). ...,appeal record sent to usca electronic file cer...,,,"Service of Process, Notices, Complaints, Lette...",0,Y
1801,NOTICE of Statement Noting a Party's Death. Do...,notice of statement noting a s death document ...,,,Notices,0,Y
1802,MANDATE of USCA (Certified Copy) as to (368 in...,mandate of usca certified copy as to in cv jsr...,,,"NoAction, Notices, Judgment, Order, Motions",1,N
1803,Costs Taxed as to (144 in 1:07-cv-09696-JSR-JL...,costs taxed as to in cv jsr jlc in cv jsr in c...,,,"NoAction, Judgment",1,N


In [233]:
print(undeteremined.shape)
undeteremined.head()

(69, 6)


,Original Docket Text,normalized,lemmed,phrases,DT Topics,NoAction
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,rule corporate disclosure statement no corpora...,rule corporate disclosure statement corporate ...,rule corporate_disclosure statement document file,,0
13,"CIVIL COVER SHEET filed. (Rosen, Samuel) (Ente...",civil cover sheet filed,civil cover sheet file,civil_cover_sheet file,,0
15,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement identifyin...,rule corporate disclosure statement identify c...,rule corporate_disclosure statement identify d...,,0
64,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0
65,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0


Apply tokenization and vectorization, then model

In [234]:
#transform the train and test data into vectors, according to countvectorizer results
CV_vect = joblib.load('Models/CountVectorizerv2.pkl')
X_CV = CV_vect.transform(undeteremined['phrases'])

TV_vect = joblib.load('Models/TfidfVectorizerv2.pkl')
X_TV = TV_vect.transform(undeteremined['phrases'])

Keras_tokenize = joblib.load('Models/Keras_tokenizev2.pkl')
X_Keras = Keras_tokenize.texts_to_matrix(undeteremined['phrases'])

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


LogisticRegression (Classifier)

In [235]:
model_name = 'Models/BM_LR'
col = 'Action LR'
loaded_model = joblib.load(model_name)

undeteremined[col + ' (Y/N)'] = loaded_model.predict(X_TV)
undeteremined[col + ' (Y/N)'] = undeteremined[col + ' (Y/N)'].map({1: 'Y', 0: 'N'})
undeteremined[col + ' prob'] = loaded_model.predict_proba(X_TV)[:, 1]

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [236]:
undeteremined.head()

,Original Docket Text,normalized,lemmed,phrases,DT Topics,NoAction,Action LR (Y/N),Action LR prob
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,rule corporate disclosure statement no corpora...,rule corporate disclosure statement corporate ...,rule corporate_disclosure statement document file,,0,Y,0.948432
13,"CIVIL COVER SHEET filed. (Rosen, Samuel) (Ente...",civil cover sheet filed,civil cover sheet file,civil_cover_sheet file,,0,N,0.064199
15,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement identifyin...,rule corporate disclosure statement identify c...,rule corporate_disclosure statement identify d...,,0,Y,0.948432
64,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0,Y,0.948432
65,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0,Y,0.948432


Adaptive Boost

In [237]:
model_name = 'Models/BM_AB'
col = 'Action AB'
loaded_model = joblib.load(model_name)

undeteremined[col + ' (Y/N)'] = loaded_model.predict(X_TV)
undeteremined[col + ' (Y/N)'] = undeteremined[col + ' (Y/N)'].map({1: 'Y', 0: 'N'})
undeteremined[col + ' prob'] = loaded_model.predict_proba(X_TV)[:, 1]

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the c

In [238]:
undeteremined.head()

,Original Docket Text,normalized,lemmed,phrases,DT Topics,NoAction,Action LR (Y/N),Action LR prob,Action AB (Y/N),Action AB prob
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,rule corporate disclosure statement no corpora...,rule corporate disclosure statement corporate ...,rule corporate_disclosure statement document file,,0,Y,0.948432,Y,0.999507
13,"CIVIL COVER SHEET filed. (Rosen, Samuel) (Ente...",civil cover sheet filed,civil cover sheet file,civil_cover_sheet file,,0,N,0.064199,N,0.001008
15,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement identifyin...,rule corporate disclosure statement identify c...,rule corporate_disclosure statement identify d...,,0,Y,0.948432,Y,0.999507
64,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0,Y,0.948432,Y,0.999507
65,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0,Y,0.948432,Y,0.999507


Keras

In [240]:
model_name = 'Models/BM_Keras.json'
model_weights = 'Models/BM_Keras.h5'
col = 'Action Keras'

json_file = open(model_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights(model_weights)

undeteremined[col + ' (Y/N)'] = np.argmax(loaded_model.predict(X_TV.toarray()), axis = 1)
undeteremined[col + ' (Y/N)'] = undeteremined[col + ' (Y/N)'].map({1: 'Y', 0: 'N'})
undeteremined[col + ' prob'] = loaded_model.predict_proba(X_TV.toarray())[:, 1]

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\framework\tensor_util.py:539: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  return np.fromstring(tensor.tensor_content, dtype=dtype).reshape(shape)
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\tensorflow\python\util\tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if d.decorator_argspec is not None), _inspect.getargspec(target))


32/69 [============>.................] - ETA: 0s

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [241]:
undeteremined.head()

,Original Docket Text,normalized,lemmed,phrases,DT Topics,NoAction,Action LR (Y/N),Action LR prob,Action AB (Y/N),Action AB prob,Action Keras (Y/N),Action Keras prob
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,rule corporate disclosure statement no corpora...,rule corporate disclosure statement corporate ...,rule corporate_disclosure statement document file,,0,Y,0.948432,Y,0.999507,Y,0.999999
13,"CIVIL COVER SHEET filed. (Rosen, Samuel) (Ente...",civil cover sheet filed,civil cover sheet file,civil_cover_sheet file,,0,N,0.064199,N,0.001008,N,0.000027
15,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement identifyin...,rule corporate disclosure statement identify c...,rule corporate_disclosure statement identify d...,,0,Y,0.948432,Y,0.999507,Y,0.999999
64,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0,Y,0.948432,Y,0.999507,Y,0.999999
65,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0,Y,0.948432,Y,0.999507,Y,0.999999


XGBoost

In [242]:
model_name = 'Models/BM_XGB'
col = 'Action XGB'
loaded_model = joblib.load(model_name)

undeteremined[col + ' (Y/N)'] = loaded_model.predict(X_TV.toarray())
undeteremined[col + ' (Y/N)'] = undeteremined[col + ' (Y/N)'].map({1: 'Y', 0: 'N'})
undeteremined[col + ' prob'] = loaded_model.predict_proba(X_TV.toarray())[:, 1]

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\sklearn\preprocessing\label.py:171: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs

In [243]:
undeteremined.head()

,Original Docket Text,normalized,lemmed,phrases,DT Topics,NoAction,Action LR (Y/N),Action LR prob,Action AB (Y/N),Action AB prob,Action Keras (Y/N),Action Keras prob,Action XGB (Y/N),Action XGB prob
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,rule corporate disclosure statement no corpora...,rule corporate disclosure statement corporate ...,rule corporate_disclosure statement document file,,0,Y,0.948432,Y,0.999507,Y,0.999999,Y,0.993116
13,"CIVIL COVER SHEET filed. (Rosen, Samuel) (Ente...",civil cover sheet filed,civil cover sheet file,civil_cover_sheet file,,0,N,0.064199,N,0.001008,N,0.000027,N,0.009948
15,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement identifyin...,rule corporate disclosure statement identify c...,rule corporate_disclosure statement identify d...,,0,Y,0.948432,Y,0.999507,Y,0.999999,Y,0.993116
64,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0,Y,0.948432,Y,0.999507,Y,0.999999,Y,0.993116
65,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0,Y,0.948432,Y,0.999507,Y,0.999999,Y,0.993116


### compare with Chris' answerkey

In [244]:
chris_answers = pd.read_excel('docket_texts/Test/DT/test_model - Oct 3 - with CH Input.xlsx', sheetname = 0)

In [245]:
print(undeteremined.shape)
print(chris_answers.shape)
chris_answers.head()

(69, 14)
(69, 11)


,Original Docket Text,normalized,lemmed,phrases,DT Topics,NoAction,Action LR,Action AB,Action Keras,Action XGB,Chris's Input (Y/N Action)
0,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,rule corporate disclosure statement no corpora...,rule corporate disclosure statement corporate ...,rule corporate_disclosure statement document f...,NaN,0,Y,Y,Y,Y,Y
1,"CIVIL COVER SHEET filed. (Rosen, Samuel) (Ente...",civil cover sheet filed entered date,civil cover sheet file enter date,civil_cover_sheet file enter date,NaN,0,N,N,N,N,N
2,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement identifyin...,rule corporate disclosure statement identify c...,rule corporate_disclosure statement identify d...,NaN,0,Y,Y,Y,Y,Y
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,NaN,0,Y,Y,Y,Y,Y
4,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,NaN,0,Y,Y,Y,Y,Y


In [246]:
temp = undeteremined.merge(chris_answers[['Original Docket Text', 'Chris\'s Input (Y/N Action)']], on = 'Original Docket Text', how = 'inner')
print(temp.shape)
temp.head()

(69, 15)


,Original Docket Text,normalized,lemmed,phrases,DT Topics,NoAction,Action LR (Y/N),Action LR prob,Action AB (Y/N),Action AB prob,Action Keras (Y/N),Action Keras prob,Action XGB (Y/N),Action XGB prob,Chris's Input (Y/N Action)
0,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. No Co...,rule corporate disclosure statement no corpora...,rule corporate disclosure statement corporate ...,rule corporate_disclosure statement document file,,0,Y,0.948432,Y,0.999507,Y,0.999999,Y,0.993116,Y
1,"CIVIL COVER SHEET filed. (Rosen, Samuel) (Ente...",civil cover sheet filed,civil cover sheet file,civil_cover_sheet file,,0,N,0.064199,N,0.001008,N,0.000027,N,0.009948,N
2,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement identifyin...,rule corporate disclosure statement identify c...,rule corporate_disclosure statement identify d...,,0,Y,0.948432,Y,0.999507,Y,0.999999,Y,0.993116,Y
3,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0,Y,0.948432,Y,0.999507,Y,0.999999,Y,0.993116,Y
4,RULE 7.1 CORPORATE DISCLOSURE STATEMENT. Ident...,rule corporate disclosure statement document f...,rule corporate disclosure statement document f...,rule corporate_disclosure statement document f...,,0,Y,0.948432,Y,0.999507,Y,0.999999,Y,0.993116,Y


In [247]:
#export and ask Chris to check
temp.to_excel('test_model_proba.xlsx', index = False)

c:\users\inves\appdata\local\programs\python\python35\lib\site-packages\pandas\io\excel.py:784: DeprecationWarning: Call to deprecated function remove_sheet (Use wb.remove(worksheet) or del wb[sheetname]).
  self.book.remove_sheet(self.book.worksheets[0])


In [248]:
temp['Action LR (Y/N)'].value_counts()

Y    55
N    14
Name: Action LR (Y/N), dtype: int64

In [249]:
temp['Chris\'s Input (Y/N Action)'].value_counts()

Y    54
N    15
Name: Chris's Input (Y/N Action), dtype: int64